## HNSW

### Get data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install faiss-cpu --no-cache

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 159.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import faiss
pd.set_option('display.max_colwidth', -1)
path = "/content/gdrive/My Drive/data"

<ipython-input-3-c4b0e23d8ed7>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [ ]:
train = pd.read_csv(path + "/gensim/ag_news_train.csv")

In [ ]:
train.shape

(120000, 3)

In [ ]:
train.head(5)

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again."
1,3,Carlyle Looks Toward Commercial Aerospace (Reuters),"Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market."
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
3,3,Iraq Halts Oil Exports from Main Southern Pipeline (Reuters),"Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday."
4,3,"Oil prices soar to all-time record, posing new menace to US economy (AFP)","AFP - Tearaway world oil prices, toppling records and straining wallets, present a new economic menace barely three months before the US presidential elections."


In [ ]:
train.columns

Index(['Class Index', 'Title', 'Description'], dtype='object')

In [ ]:
train['Description'][0:5]

0    Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.                                                                                                                        
1    Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
2    Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.                              
3    Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.                   
4    AFP - Tearaway world oil prices, toppling records and straining wallets, present a new economic menace barely three

In [ ]:
sentences = train['Description'][0:10000]

In [ ]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.5 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=76e6945bf4afe39511f2f586026749e5843c286935c88277a617b607f66764b2
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

(10000, 768)

In [ ]:
sentence_embeddings.shape[0]

10000

### Save embeddings

In [ ]:
with open(path + '/AG_news.npy', 'wb') as file:
    np.save(file, sentence_embeddings)

### Load embeddings

In [ ]:
with open (path + '/AG_news.npy', 'rb') as f:
    sentence_embeddings = np.load(f, allow_pickle=True)

In [ ]:
sentence_embeddings.shape

(10000, 768)

In [ ]:
sentence_embeddings[0:5]

array([[-0.26105028,  0.8585296 ,  0.03941074, ...,  1.0689917 ,
         1.1770816 , -0.74388623],
       [-0.2222097 , -0.03594436,  0.5209106 , ...,  0.15727971,
        -0.3867779 ,  0.49948674],
       [-0.3001758 , -0.41582862,  0.86036515, ..., -0.6246218 ,
         0.52692914, -0.36817163],
       [ 0.3295024 ,  0.22334357,  0.30229023, ..., -0.41823167,
         0.01728885, -0.05920589],
       [-0.22277102,  0.7840586 ,  0.2004052 , ..., -0.9121561 ,
         0.2918987 , -0.12284964]], dtype=float32)

In [ ]:
dim = sentence_embeddings.shape[1]
dim

768

### Build HNSW index

In [ ]:
import faiss
m = 32
dim = 768

# Creating the index.
index = faiss.IndexHNSWFlat(dim, m)
index.hnsw.efConstruction = 40         # Setting the value for efConstruction.
index.hnsw.efSearch = 16               # Setting the value for efSearch.

In [ ]:
index.add(sentence_embeddings)

In [ ]:
index.ntotal

10000

In [ ]:
# the HNSW index starts with no levels
index.hnsw.max_level

2

In [ ]:
levels = faiss.vector_to_array(index.hnsw.levels)
np.bincount(levels)

array([   0, 9713,  280,    7])

### Search example 1:

In [49]:
qry1 = model.encode(["economic booming and stock market"])

In [50]:
k=5
d, I = index.search(qry1, k)

In [51]:
print(I)

[[1467 4838 4464 7461 8299]]


In [52]:
%%time
d, I = index.search(qry1, k)
print(I)

[[1467 4838 4464 7461 8299]]
CPU times: user 9.25 ms, sys: 0 ns, total: 9.25 ms
Wall time: 8.98 ms


In [53]:
print(d)

[[158.19066 163.69077 164.47517 164.64172 164.64172]]


In [54]:
for i in I[0]:
  print(train['Description'][i])

Rising oil prices are expected to hit China's growth rate this year.
Developing countries are starting to flex their financial muscles and invest overseas.
The Tehran Stock Exchange has performed magnificently, but the market's list of risks is outsized.
Federal Express raised its earnings forecast, citing strong demand for its international express, ground and less-than-truckload services.
Federal Express raised its earnings forecast, citing strong demand for its international express, ground and less-than-truckload services.


### Search example 2:

In [ ]:
qry2 = model.encode(["Red sox won the game"])
k=5

In [ ]:
%%time
d, I = index.search(qry2, k)
print(I)

[[ 464 8517 6176 8810 8143]]
CPU times: user 9.87 ms, sys: 0 ns, total: 9.87 ms
Wall time: 13.8 ms


In [ ]:
for i in I[0]:
  print(train['Description'][i])

John Olerud reveled in his return to Safeco Field by slapping a two-run single that lifted the Yankees and saved Esteban Loaiza from a defeat.
Ted Lilly thoroughly enjoyed ending Boston #39;s six-game winning streak with a record performance. Lilly struck out a career-high 13 in a three-hitter, pitching the Blue Jays past Pedro Martinez and the Red Sox 3-0 Monday night.
SAN DIEGO (Ticker) -- Josh Beckett helped himself to his first win in over three months. Beckett doubled in the sixth inning to tie the game and scored the go-ahead run on Juan Pierre  #39;s home run one batter 
Brendan Kingman #39;s sixth-inning single sent Australia to a 1-0 victory Tuesday over stunned Japan, putting the Aussies in the gold medal game.
If they could have made the shutout last just one more inning, the unbeatable American softball team would have wound up with not only a third straight gold medal, but a feat that years from now will seem mythological.
